# The mixed method & conservation

$\newcommand{\dive}{\mathop{\mathrm{div}}}
\newcommand{\grad}{\mathop{\mathrm{grad}}}
\newcommand{\om}{\varOmega} 
\newcommand{\oh}{\varOmega_h}$

We have seen that a single boundary value problem can admit different weak formulations, possibly set in different spaces. For the Poisson equation, the *primal weak form* can be approximated by the Lagrange finite element method, while the *mixed weak form* can be approximated by a pair of $H(\dive)$ and DG finite element spaces.   This class activity is on the latter.


From the practical standpoint, one of the primary reasons for using the mixed method for the diffusion equation is *conservation*, a discrete structure-preservation property, which we shall define below.  We will use the Raviart-Thomas elements we discussed [earlier](VectorFE.ipynb) to implement and study a conservative mixed method in this notebook. 

## Two typical applications

Many boundary value problems arise by combining physical conservation laws with empirical constitutive laws. Here are two examples:

#### A. Diffusion of heat:

> Fourier's law of heat conduction is a constitutive law that states (in the absence of advective and radiative effects) that the flux of heat energy $q$ (sometime also called heat flux density) through a material is  related to the gradient of the temperature $T$ by
>
>$$    q = -\kappa \,\grad( T) $$
>
> where $\kappa$ is the measured thermal conductivity of the material. Conservation of energy implies that
>
> $$    \dive q = f $$
>
>where $f$ represents the heat source. 

#### B. Porous media flow:

>For slow viscous flow through a permeable porous medium, Darcy law is an empirical statement connecting the flux of the fluid $q$ to the pressure ($p$) gradient by 
>
>$$ q = -K \grad p $$
>
> where $K$ is the permeability tensor of the rock or the porous formation. Conservation of mass then implies 
>
> $$ \dive q = f  $$
>
> where $f$ represents injection sources, wells, or sinks. 



Historically, Model B was studied extensively by a fossil fuel industry, but in Portland, it's more likely to be applied in studying the percolation of hot water  through [coffee grinds](https://en.wikipedia.org/wiki/Darcy%27s_law#:~:text=5,in%20coffee%20brewing)!

Note that both models, after eliminating the flux results in an equation of the form  $-\dive (\alpha \grad u) = f$  which we have treated previously using Lagrange elements. In this notebook, rather than eliminating $q$, our focus is on approximating $q$.  

### Example T

Later in this notebook, we will solve a  specific example  built from Model A. The specific  problem is to simulate a steady heat flux $\newcommand{\R}{\mathbb{R}}$  $q: \om \to \R^2$ on a rectangular bar-shaped domain $\om$ of length 6 units and width 2 units, divided into equal left and right halves. More specifics of the problem appear after we draw the geometry.

In [ ]:
import ngsolve as ng
from ngsolve.webgui import Draw
from netgen.occ import X, Y, MoveTo, Rectangle, Glue, OCCGeometry
from netgen.webgui import Draw as DrawGeo

def CreateGeomOCC():
    rgtbar = Rectangle(3, 2).Face()
    lftbar = MoveTo(-3, 0).Rectangle(3, 2).Face()
    lftbar.faces.name = 'lftbar'
    rgtbar.faces.name = 'rgtbar'
    lftbar.edges.Min(X).name = 'lft'
    lftbar.edges.Min(Y).name = 'bot'
    lftbar.edges.Max(Y).name = 'top'
    rgtbar.edges.Max(X).name = 'rgt'
    rgtbar.edges.Min(Y).name = 'bot'
    rgtbar.edges.Max(Y).name = 'top'
    bar = Glue([lftbar, rgtbar])
    return bar

g = CreateGeomOCC()
DrawGeo(g);
geo = OCCGeometry(g, dim=2)
mesh = ng.Mesh(geo.GenerateMesh(maxh=1/4))

(Above, we have used the new meshing features in Netgen using [Open-Cascade](https://www.opencascade.com/). This will work only if you compiled NGsolve with OpenCascade as dependency. If it doesn't work for you, copy over the code from any of the two alternatives that I have described in [another notebook](a_Mesh3ways.ipynb).)

Here are the further specifications of the thermal problem:

- Materials whose isotropic thermal conductivity ($\kappa$) values equal 1 and 10 occupy the   left and right halves, respectively.  
- The left boundary edge of  $\Omega$ is *kept at temperature* $T=10$ while the right edge is kept  at $T=0$. 
- The top and bottom boundary edges are *perfectly insulated,*  i.e, the outward flux $q \cdot n$ vanishes there. 
- The bar is heated by a source which is modeled by 
$$
  f(x, y) = 10^3 \exp(-10 [(x/5)^2 + (y-1)^2]).
$$  

Here is a plot of $f$ and $\kappa$.

In [ ]:
from ngsolve import x, y, exp

f = 50 * exp( -10*( (x/5)**2 + (y-1)**2))
kappa = mesh.MaterialCF({'lftbar': 1, 'rgtbar': 10})
Draw(f, mesh)
Draw(kappa, mesh)

Given these, the problem is to compute the temperature $T$ and the heat flux $q$. Any guesses on which side will get heated up more in steady state?  Hold that thought; we will see if your physical intuition is right.  But before we proceed to solve for the temperature and thermal fluxes in this example, we need to see how to implement the mixed method and what we mean by exact conservation in a discrete setting.

## Conservation in the finite element context

Suppose we use a mesh $\oh$ of a $d$-dimensional domain $\om$ to compute a discrete approximation $q_h$ to the exact flux $q$. Let  $P_p(\oh) = \prod_{K \in \oh} P_p(K)$ denote the DG space, so piecewise polynomial vector fields of degree at most $p$ are in $P_p(\oh)^d.$

**Definition:** We say that a vector field $q_h$ in $P_p(\oh)^d$ is *conservative* if 
- (1) on all interior mesh facets, &LeftDoubleBracket; $q_h\cdot n$ &RightDoubleBracket; $ = 0$, and 
- (2) on every mesh element $K \in \oh$, 
$$
\int_{\partial K}  q_h \cdot n = \int_K f.
$$

In condition (1), we have used the jump notation &LeftDoubleBracket; $\cdot$ &RightDoubleBracket; introduced [earlier](VectorFE.ipynb).

To understand why  conditions (1) and (2)  are interesting properties to have, consider one of the above-mentioned applications, say the diffusion of heat. Recall that in physics,  conservation of energy is often written in *integral form* over an control volume $S$ as 
$$
\int_{\partial S} q \cdot n = \int_S f,
$$
i.e., the flux of heat leaving the subdomain $S$ through its boundary (equal to the left hand side above) must equal the amount of heat produced by sources within $V$ (equal to the right hand side above). Because this conservation statement should hold for any control volume $S$,  by the divergence theorem, the integral form results in the differential equation $\dive q = f$, which is one of the conservation equations stated previously. 

Now, consider how we may obtain a discrete version of the integral form of the conservation law. Instead of arbitrary control volumes, we restrict ourselves to the unions of the  elements used in the computation. Namely, consider  any subset of mesh elements $O_h \subseteq \oh.$ If the discrete flux out of the subdomain $S = \cup\{ K: K \in O_h\}$ formed by this subset of elements satisfies 
$$
\int_{\partial S_h}  q_h \cdot n = \int_{S_h} f,
$$
then it makes sense to declare $q_h$ to be a conservative flux, since this would be a discrete analog of the exact integral form of the conservation law, 
$
\int_{\partial S} q \cdot n = \int_S f.
$
We can immediately verify that the above equation is a consequence of conditions (1) and (2) in our definition. Note how we need *both* (1) and (2) to accomplish the interelement cancellations of influx and outflux within $S_h$.

One of the modern themes in numerical solution of partial differential equations, called  *structure preservation*, studies questions like this: how can we construct methods that (not only converge optimally, but also) yielding solutions that preserve certain critical features or structures of the exact solution? In this context, methods that produce a conservative flux $q_h$, as defined above, are structure-preserving, the preserved structure being conservation.

Another way of thinking about conservation is in terms of *superconvergence* of certain functionals. (The phenomena where the error, or some functionals of the error, goes to zero at an unexpectedly high speed is called superconvergence.) Since $f$ is related to the exact flux by $f = \dive q$, condition (2) in the definition of conservation can equivalently be written as 

$$
\int_{\partial K} q_h \cdot n  = \int_{\partial K} q\cdot n.
$$

In other words,  defining the functional $G_K (r) = \int_{\partial K} r \cdot n$, we have $G_K(q) = G_K (q_h)$.
For good methods,  we expect the error $q_h - q$ to go zero as the mesh size $h \to 0$, so 
we expect $G_K(q) - G_K(q_h) \to 0$. But it is exceptional to get  zero error for any functional. For a conservative method, this happens: $G_K(q) - G_K(q_h) = 0$.

## The Raviart-Thomas mixed method

Consider the boundary value problem of finding 
$\newcommand{\R}{\mathbb{R}}$
$u: \om\to \R$ and $q: \om \to \R^d$ ($d\ge 2$), given $f: \om \to \R$ and (pointwise) invertible $\kappa : \om \to \R^{d \times d}$, satisfying 

$$
\kappa^{-1} q + \grad u =0, \qquad \dive q = f, 
$$

with Dirichlet boundary conditions $u =0 $ on $\partial \om$. From the lectures, we know that the **mixed weak formulation** of this problem reads as follows.

Find $u \in L^2(\om)$ and $q \in H(\dive, \om)$ satisfying 

$$
\begin{aligned}
\int_\om \kappa^{-1} q \cdot r - \int_\om u\, \dive r & = 0, 
&& \text{ for all } r \in H(\dive, \om), \text{ and }
\\
\int_\om v\, \dive q & = \int_\om f \,v,  
&& \text{ for all } v \in L^2(\om).
\end{aligned}
$$


To obtain a numerical method, starting from the above weak form, 
we use the  DG space $P_p(\oh)$ in place of $L^2(\om)$ and the Raviart-Thomas finite element space (introduced [earlier](VectorFE.ipynb)) 
$$
R_{h, p} =
\left\{
q\in H(\dive, \om) : \quad \forall K \in \oh, \quad  q|_K \in P_p(K)^2 + \begin{pmatrix} x \\ y \end{pmatrix} P_p(K) \; \right\}
$$
in place of $H(\dive, \om).$ Recall that we have already proved that $R_{h, p} \subset H(\dive, \om).$ 

The **Raviart-Thomas mixed method** of order $p$ finds $u_h \in  P_p(\oh)$ and
$q \in R_{h, p}$ satisfying 

$$
\begin{aligned}
\int_\om \kappa^{-1} q_h \cdot r_h - \int_\om u_h\, \dive r_h & = 0, 
&& \text{ for all } r_h \in R_{h, p}, \text{ and }
\\
\int_\om v_h\, \dive q_h & = \int_\om f \,v_h,  
&& \text{ for all } v_h \in P_p(\oh).
\end{aligned}
$$

In the latter equation, if we select a test function $v_h$ that is the indicator function of one element $K \in \oh$, which is of course contained in $P_p(\oh)$ for any $p \ge 0$, then we obtain 

$$
\int_K \dive q_h = \int_K f, 
$$

the second condition in the definition of a conservative flux. Of course, condition (1) is automatically satisfied by $q_h$, since every $q_h \in R_{h, p}$ has 
&LeftDoubleBracket; $q_h\cdot n$ &RightDoubleBracket; $ = 0$. Thus *this mixed method yields conservative fluxes.* In NGSolve, there are two ways to assemble such a mixed weak form, as we shall see in the next section. 

## Two ways to assemble
 
 

#### The first way: assemble a composite form

We can  think of the system using a "big" composite bilinear form $C(\cdot, \cdot)$ in the product space $R_{h, p} \times P_p(\oh)$, i.e., for any  $(q_h, u_h), (r_h, v_h) \in 
R_{h, p} \times P_p(\oh)$, set 

$$
C((q_h, u_h), (r_h, v_h)) = 
\int_\Omega \kappa^{-1} q_h \cdot r_h - \int_\Omega u_h \,\text{div } r_h
- \int_\Omega v_h \,\text{div } q_h.
$$

Then the previous set of two equations for the Raviart-Thomas mixed method can be described as the single equation

$$
C((q_h, u_h), (r_h, v_h)) = \int_\om f v_h,
\qquad \text{ for all } v_h \in P_p(\oh), \, r_h \in R_{h,p}.
$$

To compute with this form, we need the product space $R_{h, p} \times P_p(\oh)$. This space is represented by the object `RW` introduced below. 

In [ ]:
import ngsolve as ng
p = 4 
R = ng.HDiv(mesh, order=0, RT=True)
W = ng.L2(mesh, order=0)
RW = ng.FESpace([R, W])

Note that trial and test functions in `RW` have two components. Otherwise the procedure for declaring  the bilinear form is exactly the same as what we saw previously. At the end of the code below, we have a sparse assembled matrix representing the stiffness matrix of the big bilinear form $C(\cdot, \cdot)$ on the product finite element space.

In [ ]:
from ngsolve import div, dx

q, u = RW.TrialFunction() 
r, v = RW.TestFunction()
C = ng.BilinearForm(RW)
C += ((1/kappa) * q*r - u*div(r) - div(q)*v) * dx
C.Assemble()
type(C.mat)

#### The second way: assemble a block system

In the second approach, we make individual components of the biliear form and put them together, i.e.,  we make the following "small" bilinear forms:

$$
a(q, r) = \int_\Omega q \cdot r, \qquad
b(q, v) = -\int_\Omega v \,\text{div } q
$$

Then the equations RT mixed method can be rewritten using these forms as 

$$
\begin{aligned}
a(q_h, r_h) + b(r_h, v) & = 0 
&& \text{ for all } r_h \in R_{h, p}, \text{ and }
\\
\,b( q_h, v_h)  & = -\int_\om f \,v_h,  
&& \text{ for all } v_h \in P_p(\oh).
\end{aligned}
$$

Note that the bilinear form $b$  uses different trial and test space. Implementing this requires the use of keyword arguments `trialspace` and `testspace` that we have not seen previously, but is self explanatory. There is no need to define a product space in this approach. 

In [ ]:
q, r = R.TnT()
u, v = W.TnT()

b = ng.BilinearForm(trialspace=R, testspace=W)
b += -div(q)*v * dx

a = ng.BilinearForm(R)
a += (1/kappa) * q*r * dx

b.Assemble(); a.Assemble();

When you make such component forms, you also need a facility to place them into appropriate places of a block partitioned matrix. Let $\mathtt A$ and $\mathtt B$  denote the stiffness matrices of the forms $a(\cdot, \cdot)$ and $b(\cdot, \cdot)$ made in the code block above, respectively, then the matrix object `BB` made below represents 
$$
\mathbb B = 
\begin{bmatrix}
\mathtt A & \mathtt B^T \\ 
\mathtt B & 0 
\end{bmatrix}
$$
where ${}^T$ denotes the transpose. The `None` element in the code below is how we represent  the zero block. 

In [ ]:
BB = ng.BlockMatrix([[a.mat, b.mat.T], 
                     [b.mat, None]])

Obviously $\mathbb B$ and $C$ represent the same linear operator. Block structures are just a convenience feature to work with components (without replicating their already allocated memory).  Block matrices expect block vectors when asked to perform matrix multiplication. I'll warn you that it's not a good idea to directly perform a generic vector operation between a block vector object and a regular NGSolve vector. (In some cases NGSolve will raise an exception and will not allow you to do so.) Also, when you work with block vectors, it is a good idea to ensure that you have variable names bound to its individual component vectors in your python workspace.  Here is an exercise to get some practice on working with the `BlockMatrix` structures and with matrix-vector products in NGSolve. 

**Exercise 1:**  Write a python code to verify that the application of `BB` and `C` 
(which are different NGSolve objects of type  `BlockMatrix` and `SparseMatrixd`, respectively, representing the same linear operator) to the same vector gives the same result.  (You will have to look up some NGSolve structures we have not used, e.g., the `component` attribute of product space grid functions.)

## Solving for thermal flux 

We now return to solve Example T stated earlier. Translating the task to that of solving a boundary value problem, we need to solve for temperature $T$ satisfying 
$$\newcommand{\lft}{\partial\om_{\text{left}}}
\newcommand{\rgt}{\partial\om_{\text{right}}}
\begin{aligned}
\kappa^{-1} q + \grad T & = 0, && \text{ in } \om
\\
\dive q & = 10^3 e^{-10 [(x+1)^2 + (y-1/2)^2]},  && \text{ in } \om
\\
q\cdot n & = 0 && \text{ on top and bottom boundaries} 
\\
T & = 100  && \text{ on the left boundary } \lft 
\\
T & = 0 && \text{ on the right boundary } \rgt.
\end{aligned}
$$

In [ ]:
f = 50 * exp( -10*( (x/5)**2 + (y-1)**2))
kappa = mesh.MaterialCF({'lftbar': 1, 'rgtbar': 10})
Tbdr = mesh.BoundaryCF({'lft': 10, 'rgt': 1})

To solve this using the mixed method, we have to carefully incorporate the boundary conditions into the mixed formulation. From previous lectures, we know that Dirichlet and Neumann conditions are respectively imposed as natural and essential conditions in the mixed formulation, in  *exactly the opposite* manner of the primal formulation. Hence we use the subspace of the Raviart-Thomas space where the flux boundary conditions are incorporated essentially:

$$
\newcommand{\Rtb}{\mathring{R}_{h, p}^{\text{top, bot}}}
\Rtb= \{ r \in R_{h, p}: r\cdot n = 0 \text{ on the top and bottom boundaries}\}.
$$

Also incorporating the Dirichlet boundary conditions on $T$ naturally, we obtain the following equations of the method: find $T_h \in P_p(\oh)$ and $q_h \in \Rtb$ satisfying 

$$
\begin{aligned}
\int_\om \kappa^{-1} q_h \cdot r_h - \int_\om T_h\, \dive r_h & = -\int_{\lft} 100\, r_h \cdot n, 
&& \text{ for all } r_h \in \Rtb \text{ and }
\\
\int_\om v_h\, \dive q_h & = \int_\om f \,v_h,  
&& \text{ for all } v_h \in P_p(\oh).
\end{aligned}
$$

#### Block linear system 

We adopt the second approach to assembly mentioned above, to produce a block linear system, whose right and left hand sides are returned by the next function. To make the right hand side of the first equation, we  need to integrate  the trace of $r_h \cdot n$ along boundary edges, which is accomplished by `r.Trace() * n`  and `ds` provided by NGSolve.

In [ ]:
from ngsolve import dx, ds

def MakeMixed(mesh, f, Tbdr, kappa, p=4):    
    """Return the block linear system (rhs and lhs) of the
    RT mixed method for solving Example T"""
    
    R = ng.HDiv(mesh, order=p, RT=True, dirichlet='top|bot')
    W = ng.L2(mesh, order=p)
    q, r = R.TnT()
    u, v = W.TnT()
    dl = ds(definedon=mesh.Boundaries('lft|rgt'))
    n = ng.specialcf.normal(mesh.dim)

    b = ng.BilinearForm(trialspace=R, testspace=W)
    b += -div(q) * v * dx
    a = ng.BilinearForm(R)
    a += (1/kappa) * q * r * dx    
    fR = ng.LinearForm(R)
    fR += -Tbdr * (r.Trace() * n) * dl
    fW = ng.LinearForm(W)
    fW += -f * v * dx
    with ng.TaskManager():
        b.Assemble() 
        a.Assemble()
        fR.Assemble()
        fW.Assemble() 
    BB = ng.BlockMatrix([[a.mat, b.mat.T], [b.mat, None]])
    FF = ng.BlockVector([fR.vec, fW.vec])   
    return BB, FF, R, W

In [ ]:
BB, FF, R, W = MakeMixed(mesh, f, Tbdr, kappa)

Next, we need to solve for a vector `x` satisfying the block system `BB * x = FF`.  All the similar previous cases involved assembled sparse matrix objects which we could hand over to a sparse solver (using the `Inverse(...)` method). However, the block matrix `BB` does not have a sparse  `Inverse` method.  This is a good opportunity to consider iterative solvers. 

#### Iterative solver

Iterative solvers built using Krylov spaces (instead of an input matrix) only require an input matrix-like object that can perform matrix-vector multiplication. The block matrix `BB` can perform the multiplication `BB * x`.  Although Krylov space iterative solvers can be easily implemented in a few lines of python code, let us make use of a readymade Krylov solver implementation that NGSolve provides. **Minres** is an iterative method suitable for invertible symmetric (not necessarily positive definite) linear systems, like the block system we have produced. Its implementation is available in `ng.solvers.MinRes`.

The rate of convergence of iterative solvers like Minres depends on the condition number of the system. Hence they are usually effective only when we also provide it a good preconditioner. A preconditioner for use in solving $\mathbb B x = b$ is a linear operator $\mathbb P$ (acting on the same-size vectors as $\mathbb B$) with the property that $\mathbb P \mathbb B$ is better conditioned than $\mathbb B$.  Obviously, $\mathbb P = \mathbb B^{-1}$ achieves this property, but we also want $\mathbb P$ to be inexpensive to apply. Once a preconditioner is given, the iterative solver, instead of solving $\mathbb B x = b$, solves the equivalent but better conditioned system $\mathbb P\mathbb B x = \mathbb P b$ behind the scenes.

Of course, one could just set $\mathbb P$ to the identity operator on the free degrees of freedom to see the performance of solver without preconditioning (i.e., with $\mathbb P$ equal to identity).

In [ ]:
identityR = ng.Projector(mask=R.FreeDofs(), range=True)
identityW = ng.IdentityMatrix(W.ndof)
PI = ng.BlockMatrix([[identityR, None],
                     [None, identityW]])

In [ ]:
qT = ng.BlockVector([ng.GridFunction(R).vec, 
                     ng.GridFunction(W).vec])
ng.solvers.MinRes(mat=BB, pre=PI, rhs=FF, sol=qT, maxsteps=15);

As you can see the convergence is abysmally slow. Feel free to increase `maxsteps` and run this again to see how many iterations you might need to do before getting anywhere close to convergence.

We need a better preconditioner.  For our block matrix 
$$
\mathbb B = 
\begin{bmatrix}
\mathtt A & \mathtt B^T \\ 
\mathtt B & 0 
\end{bmatrix},
$$
one technique is to construct a preconditioner in the following form, using the same block partitioning,
$$
\mathbb P = 
\begin{bmatrix}
\mathtt M_R^{-1} & 0 \\
0  & \mathtt M_W^{-1}
\end{bmatrix},
$$
where $\mathtt M_R$ and $\mathtt M_W$ are the stiffness matrices of the bilinear forms 
$$
\int_\om \kappa^{-1} q\cdot r + \int_\om (\dive q ) (\dive r), \qquad q, r \in \Rtb
$$
and 
$$
\int_\om u \, v, \qquad u, v \in P_p(\oh),
$$
respectively. Since $\mathtt M_W$ is block diagonal with one block for each element (why?), it is very easy to invert. So the cost of construction of this preconditioner is essentially the cost of inversion of $\mathtt M_R$,  a smaller matrix than the original $\mathbb B$.

In [ ]:
def MakePrec(R, W):
    q, r = R.TnT()
    mR = ng.BilinearForm(((1/kappa)*q*r + div(q)*div(r))*dx)
    mR.Assemble()
    P = ng.BlockMatrix([[mR.mat.Inverse(R.FreeDofs()), None],
                        [None,          W.Mass(1).Inverse()]])
    return P

In [ ]:
PP = MakePrec(R, W)

In [ ]:
qh = ng.GridFunction(R)
Th = ng.GridFunction(W)
qT = ng.BlockVector([qh.vec, Th.vec])
ng.solvers.MinRes(mat=BB, pre=PP, rhs=FF, sol=qT, maxsteps=15);

Why this preconditioner works is an interesting question that we will postpone for another time. For now, let's settle for visualizing the computed solution.

#### Temperature and flux 


The solution is overwritten in the same memory location as the input `qT` by Minres. This is in the form of a block vector, whose components occupy the same memory as `qh` and `Th`. So to plot the temperature, we can directly use the variable name `Th`.

In [ ]:
Draw(Th);

This plot of the calculated temperature $T$ reveals lesses temperature variations on the right than on the left. In fact, if you turn on deformation in the webgui above, you will see that there is a kink at the middle interface (indicating a discontinuity in $\grad T$). We also see that even though the blob of heat source was centered in the domain, it resulted in an uncentered temperature peak to the left.  *Is this shift of the peak due to the difference in boundary conditions on the left and right, or due to the difference in the left and right thermal conductivity?*   The utility of having a simulation tool is that we can now answer questions like this easily. Go ahead and experiment, changing the boundary conditions, and changing the thermal conductivites set in the code cells above as you please.  You will then be led to the conclusion that the shift of the peak is primarily due to the fact that the right half of the domain, due to its higher thermal conductivity, can transmit heat through it faster thus producing lesser variations in the temperature profile there. Plotting the computed thermal fluxes will further confirm this:

In [ ]:
Draw(qh, vectors={'grid_size': 35});    

Heat flow on the right is more rapid  than on the left. (The colors represent the magnitude of the heat flux.) In fact, from the direction of the fluxes, we see that the left boundary condition is helping to dissipate some of the heat generated by the source. Also observe the apparent continuity of the normal component of the heat flux despite the jump in the material properties. This is exactly as it should be, as otherwise energy creation or energy loss will be predicted across the interface, which would be nonphysical (as energy must be conserved).




**Exercise 2.** Implement a solver for Example T using Lagrange finite elements applied to the primal weak form.  Using your computed temperature $\tilde T_h$ (in the Lagrange space $ V_{h, p}$) from this approach, further compute an approximate flux $\tilde{q}_h = -\kappa \grad \tilde{T}_h$. Computationally show that this flux is not conservative. Specifically, letting $\Gamma$ denote the middle interface, compute 
$\int_\Gamma$ 
&LeftDoubleBracket; $\tilde q_h\cdot n$ &RightDoubleBracket; to show that it is nonzero. For comparison, letting $q_h$ denote the flux from the Raviart-Thomas mixed method, compute 
$\int_\Gamma$ 
&LeftDoubleBracket; $q_h\cdot n$ &RightDoubleBracket; and show that it is close to machine zero. (Research online examples and tutorials to learn of code options for computing such integrals correctly.)